In [7]:
# list import
list_path = 'api.nal.usda.gov/ndb/list'
api_key = '{HIDDEN}'
fmt = 'json'
items_max = 143
headers = {'Content-Type': 'application/{fmt}'.format(fmt=fmt)}

In [4]:
params = {
    'lt': 'f',
    'sort': 'n',
    'max': items_max,
    'format': fmt,
    'offset': 259000,
    'api_key': api_key
}
req = requests.get('http://{list_path}'.format(list_path=list_path), headers=headers, params=params)
req.raise_for_status()


In [36]:
req.json()
req

<Response [200]>

In [8]:
import time
import requests

offset = 0
offset_shift = 500
failed = []
success = set()

with open('ids.data', 'a') as fname:
    while True:
        params = {
            'lt': 'f',
            'sort': 'n',
            'max': items_max,
            'format': fmt,
            'offset': {offset},
            'api_key': api_key
        }
        try:
            req = requests.get('http://{list_path}'.format(list_path=list_path), headers=headers, params=params)
            req.raise_for_status()
            if len(req.json()['list']['item']) < offset_shift:
                raise Exception('Chunk len mismatch')
            if req.json()['list']['start'] != offset:
                print 'finished at %s' % offset
                break
            offset = offset + items_max
            time.sleep(1.1)
        except:
            print 'failed'
            failed.append(offset)
            break
        else:
            len_before = len(success)
            for key in req.json()['list']['item']:
                success.add(key['id'])
                fname.write(key['id']+'\n')
            len_after = len(success)
            if len_before == len_after:
                print 'len does not changed'
                break
            print len(success)


143
failed


In [3]:
print len(success)
print len(failed)

259000
1


In [ ]:
offset_shift = 25
failed = []
ids_to_go = []
break_ = False
import json

report_v2_path = 'api.nal.usda.gov/usda/ndb/V2/reports'
with open('report_full.data', 'a') as w_fname:
    with open('ids_full.data', 'r') as ids_fname:
        while not break_:
            ids_to_go = []
            while len(ids_to_go) < 24:
                id_ = ids_fname.readline()
                ids_to_go.append(id_.strip('\n'))
            params = {
                "ndbno":ids_to_go,
                "type":"b",
                'format': fmt,
                'api_key': api_key
            }
            try:
                req = requests.get('http://{report_v2_path}'.format(report_v2_path=report_v2_path), headers=headers, params=params)
                req.raise_for_status()
            except Exception as exc:
                #break_ = True
                print exc
                failed.append(ids_to_go)
            else:
                for item in req.json()['foods']:
                    w_fname.write(json.dumps(item))
                    w_fname.write('\n')
            time.sleep(1.1)